In [7]:
# from flask import Flask, jsonify, request, render_template
from flask import Flask
import joblib
import pandas as pd
import numpy as np
import shap
import json
import matplotlib.pyplot as plt
import streamlit as st
import streamlit.components.v1 as components
import plotly.express as px
import plotly
import pickle
import os
import lightgbm

app = Flask(__name__)
print(pd.__version__)
current_folder = os.getcwd()
basepath = os.path.join(current_folder, "Models")

# load models, threshold, data and explainer
model_load = joblib.load(os.path.join(basepath, "model.joblib"))
best_thresh = joblib.load(os.path.join(basepath, "best_thresh_LightGBM_NS.joblib"))
X_test = joblib.load(os.path.join(basepath, "X_test.pkl"))
y_test = joblib.load(os.path.join(basepath, "y_test.pkl"))
y_pred = joblib.load(os.path.join(basepath, "y_pred.pkl"))
explainer = joblib.load(os.path.join(basepath, "explainer"))
shap_values = joblib.load(os.path.join(basepath, "shap_values.pkl"))
shap_values1 = joblib.load(os.path.join(basepath, "shap_values1.pkl"))
expected_value = joblib.load(os.path.join(basepath, "expected_values.pkl"))
columns = shap_values.feature_names
data = pd.DataFrame(y_test, index=y_test.index).reset_index()
#data["PRED"] = y_pred

# Compute feature importance
# compute mean of absolute values for shap values
vals = np.abs(shap_values1).mean(0)
# compute feature importance as a dataframe containing vals
feature_importance = pd.DataFrame(list(zip(columns, vals)),\
    columns=['col_name','feature_importance_vals'])
# Define top 10 features for customer details
top_10 = feature_importance.sort_values(by='feature_importance_vals', ascending=False)[0:10].col_name.tolist()
# Define top 20 features for comparison vs group
top_20 = feature_importance.sort_values(by='feature_importance_vals', ascending=False)[0:20].col_name.tolist()
feat_tot = feature_importance.feature_importance_vals.sum()
feat_top = feature_importance.loc[feature_importance['col_name'].isin(top_20)].feature_importance_vals.sum()

@app.route("/", methods=['GET']) #hello_world() sera appelée lorsque l'utilisateur accède à la racine de l'application (l'URL /)"
def hello_world():
    return "<p>Hello, World!</p>"


@app.route("/predict/<int:Client_Id>", methods=['GET'])
def predict(Client_Id: int):
    # Customer index in the corresponding array
    data_idx = data.loc[data["SK_ID_CURR"]==int(Client_Id)].index[0]
    # Customer data based on customer index in final X_test array
    ID_to_predict = pd.DataFrame(X_test.iloc[data_idx,:]).T  
    # on réalise la prédiction de ID_to_predict avec le modèle 
    prediction = sum((model_load.predict_proba(ID_to_predict)[:, 1]>best_thresh)*1)
    if prediction == 0:
        decision = "granted"
    else :
        decision = "not granted"
    prob_predict = float(model_load.predict_proba(ID_to_predict)[:, 1])
    # on renvoie la prédiction 
    return json.dumps({"decision" : decision, "prediction" : int(prediction), 
                       "prob_predict": prob_predict, "ID_to_predict" : ID_to_predict.to_json(orient='columns')})

# provide data for shap features importance on selected customer's credit decision 
@app.route("/cust_vs_group/<int:Client_Id>", methods=['GET'])
def cust_vs_group(Client_Id: int):
    # utiliser idx pour former le graph via Flask et l'importer dans streamlit
    data_idx = data.loc[data["SK_ID_CURR"]==int(Client_Id)].index[0] #string ou pas
    # customer data based on customer index in final X_test array
    ID_to_predict = pd.DataFrame(X_test.iloc[data_idx,:]).T
    # on réalise la prédiction de ID_to_predict avec le modèle 
    prediction = sum((model_load.predict_proba(ID_to_predict)[:, 1]>best_thresh)*1)
    if prediction == 0:
        decision = "granted"
    else :
        decision = "not granted"
    # return json string
    return json.dumps({'decision' : decision, 'base_value': shap_values.base_values[data_idx], 
                       'shap_values1_idx': shap_values1[data_idx, :].tolist(), \
    "ID_to_predict": ID_to_predict.to_json(orient='columns')})


@app.route("/load_top_10/", methods=['GET'])
def load_top_10():
    return json.dumps({"top_10" : top_10})

@app.route("/load_top_20/", methods=['GET'])
def load_top_20():
    return json.dumps({"top_20" : top_20, 'feat_tot': feat_tot, 'feat_top': feat_top})

@app.route("/load_best_thresh/", methods=['GET'])
def load_best_thresh():
    return {"best_thresh" : best_thresh} 

@app.route("/load_X_test/", methods=['GET'])
def load_X_test():
    return {"X_test" : pd.DataFrame(X_test).to_json(orient='columns')} 

@app.route("/load_data/", methods=['GET'])
def load_data():
    return {"data" : pd.DataFrame(data).to_json(orient='columns')} 

if __name__ == "__main__":
    print("Starting server on port 8500")
    print("Running...")
    
    app.run(port=8500,debug=True , use_reloader=False)
    # app.run(host="localhost", port=5000,debug=True , use_reloader=False) 
    #modification 1 --> app.run(port=5001, debug=True), %tb
#app.run(host="0.0.0.0", port=5000,debug=True , use_reloader=False)
    print("Stopped")
#CODE A planté --> le host=localhost --> app.run(host="localhost", port=2000,debug=True , use_reloader=False)

1.5.3
Starting server on port 8500
Running...
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8500
Press CTRL+C to quit
127.0.0.1 - - [17/Jun/2024 11:33:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2024 11:33:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Jun/2024 11:33:20] "GET /load_top_10 HTTP/1.1" 308 -
127.0.0.1 - - [17/Jun/2024 11:33:20] "GET /load_top_10/ HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2024 11:33:37] "GET /predict/307488 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2024 11:35:18] "GET /predict/326926 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2024 11:41:21] "GET /predict/307488 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2024 11:44:34] "GET /predict/345565 HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 2548, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.handle_exception(e)
  File "C:\ProgramData\anaconda3\lib\site-packages\flask\app.py", line

Stopped


In [21]:
y_test

,TARGET
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
58257,0.0
58258,0.0
58259,0.0
58260,0.0


In [2]:
X_test.head()

,CODE_GENDER,AMT_CREDIT,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,REGION_RATING_CLIENT,...,APPROVED_HOUR_APPR_PROCESS_START_MEAN,APPROVED_RATE_DOWN_PAYMENT_MAX,APPROVED_DAYS_DECISION_MIN,APPROVED_DAYS_DECISION_MEAN,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_COUNT,INSTAL_DBD_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM
0,1.0,521280.0,450000.0,0.008575,-15323.0,-1503.0,-2574.0,-3753.0,1.0,2.0,...,14.00,0.265029,-1863.0,-1863.00,-58.00000,7.0,7.0,42.0,-1753.833333,-10523.0
1,0.0,1185120.0,900000.0,0.046220,-15860.0,-356.0,-7002.0,-1683.0,1.0,1.0,...,11.00,0.000000,-321.0,-321.00,-7.50000,6.0,6.0,83.0,-246.600000,-1233.0
2,0.0,508495.5,454500.0,0.008474,-20277.0,-384.0,-4278.0,-1904.0,1.0,2.0,...,16.00,0.417055,-2719.0,-2719.00,-84.50000,10.0,10.0,145.0,-2584.111111,-23257.0
3,1.0,378000.0,378000.0,0.026392,-19401.0,-1861.0,-9256.0,-2950.0,1.0,2.0,...,11.00,0.104286,-568.0,-460.50,-10.60000,20.0,20.0,778.0,-1760.677966,-207760.0
4,1.0,113746.5,103500.0,0.007120,-13565.0,-1151.0,-2293.0,-756.0,1.0,2.0,...,11.25,0.132410,-2533.0,-1213.25,-39.90625,32.0,32.0,145.0,-1161.727273,-38337.0


In [11]:
import sys
print(sys.version)

3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]


In [12]:
import sys
print(sys.version)
print(get_ipython().__class__.__name__)

3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]
ZMQInteractiveShell
